In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats

tips = sns.load_dataset("tips")
titanic = sns.load_dataset('titanic')
titanic.dropna(inplace=True)
penguins = sns.load_dataset("penguins")
iris = sns.load_dataset('iris')
dia = sns.load_dataset("diamonds")


문제1. price를 종속변수로 하고 나머지 수치형 변수를 독립변수로 했을때 다중 공선성의 문제가 있다고 판단되는 변수의 개수는?

In [2]:
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

In [6]:
dia

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [13]:
y,X =dmatrices('price~carat+depth+table+x+y+z',
          data=dia,
          return_type='dataframe')

dia_vif = pd.DataFrame() #빈 데이터 프레임 생성
dia_vif['colname'] = X.columns
dia_vif['VIF'] = [vif(X.values,i) for i in range(X.shape[1])]

In [15]:
dia_vif #답 4개

,colname,VIF
0,Intercept,4821.696350
1,carat,21.602712
2,depth,1.496590
3,table,1.143225
4,x,56.187704
5,y,20.454295
6,z,23.530049


문제2. price 종속변수 ,carat과 depth 독립변수 일떄 carat =1, depth=60, table=55 예측

In [20]:
dia

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [18]:
from sklearn.linear_model import LinearRegression

In [19]:
model = LinearRegression()

In [21]:
model.fit(X=dia[['carat','depth']],y=dia['price'])

LinearRegression()

In [40]:
x_test = pd.DataFrame({'carat':1, 'depth':60},index=[0])

In [41]:
y_pred=model.predict(x_test)

In [42]:
y_pred

array([5680.55451689])

문제3. price를 종속변수로 하고 carat, color, depth를 독립변수로둔 선형회귀모델로 carat:1, depth:50, color:E 예측

In [44]:
from statsmodels.formula.api import ols

In [45]:
model= ols('price~carat+C(color)+depth',data=dia).fit()

In [46]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.865
Model:                            OLS   Adj. R-squared:                  0.865
Method:                 Least Squares   F-statistic:                 4.319e+04
Date:                Fri, 22 Apr 2022   Prob (F-statistic):               0.00
Time:                        01:58:06   Log-Likelihood:            -4.6977e+05
No. Observations:               53940   AIC:                         9.396e+05
Df Residuals:                   53931   BIC:                         9.396e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      3399.1860    272.825     12.459      0.000    2864.447    3933.925
C(color)[T.E]   -97.0161     23.164     -4.188      0.000    -142.417     -51.615
C(color)[T.F]   -80.8972     23.316     -3.470      0.001    -126.596     -35.199
C(color)[T.G]   -80.6971     22.585     -3.573      0.000    -124.963     -36.431
C(color)[T.H]  -720.8099     24.268    -29.703      0.000    -768.374    -673.245
C(color)[T.I] -1043.9064     27.213    -38.361      0.000   -1097.243    -990.570
C(color)[T.J] -1899.5248     33.657    -56.438      0.000   -1965.493   -1833.557
carat          8070.6389     13.988    576.983      0.000    8043.223    8098.055
depth           -89.7605      4.412    -20.344      0.000     -98.408     -81.113
==============================================================================
Omnibus:                    12411.519   Durbin-Watson:                   0.953
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           159901.705
Skew:                           0.746   Prob(JB):                         0.00
Kurtosis:                      11.302   Cond. No.                     2.67e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.67e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [51]:
dia_test = pd.DataFrame({'carat':1,'depth':50,'color':"E"},index=[0])

In [52]:
dia_test

,carat,depth,color
0,1,50,E


In [53]:
model.predict(dia_test)

0    6884.782287
dtype: float64